# 하이퍼파라미터 튜닝: GridSearchCV를 이용한 최적 모델 찾기

이 노트북은 머신러닝 모델의 성능을 최적화하기 위한 핵심 과정인 **하이퍼파라미터 튜닝(Hyperparameter Tuning)**을 다룹니다. 특히 `scikit-learn`의 `GridSearchCV`를 사용하여 여러 모델(`SVC`, `RandomForestClassifier`, `GradientBoostingClassifier`)에 대한 최적의 하이퍼파라미터 조합을 찾는 방법을 보여줍니다.

### 하이퍼파라미터 튜닝이란?
하이퍼파라미터는 모델 학습 과정에서 사용자가 직접 설정해야 하는 값들입니다(예: 트리의 깊이, 규제 강도). 이 값들은 모델의 성능에 큰 영향을 미치므로, 최적의 조합을 찾는 것이 중요합니다.

### GridSearchCV란?
`GridSearchCV`는 사용자가 지정한 하이퍼파라미터 값들의 모든 가능한 조합에 대해 모델을 학습시키고 교차 검증(Cross-Validation)을 수행하여, 가장 좋은 성능을 보이는 조합을 찾아주는 도구입니다.

### 1. 라이브러리 임포트 및 데이터 로드

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_breast_cancer 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score 

# 유방암 데이터셋 로드
cancer = load_breast_cancer() 
X = cancer.data 
y = cancer.target 

print(f"데이터 형태: {X.shape}")
print(f"타겟 형태: {y.shape}")

### 2. 데이터 분할

모델 학습과 평가를 위해 데이터를 훈련 세트와 테스트 세트로 분할합니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234)

print(f"훈련 데이터 형태: {X_train.shape}")
print(f"테스트 데이터 형태: {X_test.shape}")

### 3. 하이퍼파라미터 그리드 정의

각 모델에 대해 탐색할 하이퍼파라미터와 그 값의 범위를 딕셔너리 형태로 정의합니다.

In [ ]:
param_grid = {
    'svc':{
        'C':[0.1, 1, 10, 100], # 규제 강도: 값이 클수록 모델이 훈련 데이터에 더 적합하려 함 (과대적합 위험)
        'gamma':[1, 0.1, 0.01, 0.001], # 커널의 영향 범위: 값이 클수록 하나의 훈련 샘플이 미치는 영향이 커짐 (과대적합 위험)
        'kernel':['rbf', 'linear'] # 커널 함수: 'rbf'는 비선형, 'linear'는 선형 분류에 사용
    }, 
    'random_forest':{
        'n_estimators':[50, 100, 200], # 트리의 개수
        'max_depth':[None, 3, 10, 20], # 트리의 최대 깊이 (None은 제한 없음)
        'min_samples_split':[2,5,10] # 노드를 분할하기 위한 최소 샘플 수
    },
    'gradient_boosting':{
        'n_estimators':[50, 100, 200], # 트리의 개수
        'max_depth':[3, 5, 10], # 트리의 최대 깊이
        'learning_rate':[0.01, 0.1, 0.2] # 학습률: 각 트리가 이전 트리의 오차를 얼마나 강하게 보정할지 결정
    }
}

### 4. GridSearchCV 실행

각 모델에 대해 정의된 하이퍼파라미터 그리드를 탐색하여 최적의 조합을 찾습니다.

- `estimator`: 튜닝할 모델 객체
- `param_grid`: 탐색할 하이퍼파라미터 그리드
- `cv`: 교차 검증 폴드 수 (예: 5는 5-fold 교차 검증)
- `verbose`: 진행 상황 출력 레벨
- `scoring`: 모델 평가 지표 (예: 'accuracy')

In [ ]:
results = {}
models = {
    'svc': SVC(random_state=42),
    'random_forest': RandomForestClassifier(random_state=42),
    'gradient_boosting': GradientBoostingClassifier(random_state=42)
}

for model_name, model in models.items():
    print(f"
Running GridSearchCV for {model_name}...")
    grid = GridSearchCV(estimator=model, 
                        param_grid=param_grid[model_name], 
                        cv=5, 
                        verbose=1, 
                        scoring='accuracy',
                        n_jobs=-1) # 모든 CPU 코어 사용
    grid.fit(X_train, y_train)
    results[model_name] = {
        'best_params': grid.best_params_,
        'best_score': grid.best_score_,
        'best_model': grid.best_estimator_
    }

### 5. 테스트 데이터에 대한 성능 평가

`GridSearchCV`를 통해 찾아낸 각 모델의 최적 하이퍼파라미터 조합으로 학습된 모델을 사용하여, 테스트 데이터에 대한 최종 성능을 평가합니다.

In [ ]:
for model_name, result in results.items():
    print(f"
--- Model: {model_name} ---")
    print("Best Parameters (최적 하이퍼파라미터):", result['best_params'])
    print("Best Cross-Validation Score (최적 교차 검증 점수):", f"{result['best_score']:.4f}")
    
    # 테스트 데이터로 예측
    y_pred = result['best_model'].predict(X_test)
    
    print("
Classification Report (분류 리포트):
", classification_report(y_test, y_pred))
    print("Test Set Accuracy (테스트 세트 정확도):", f"{accuracy_score(y_test, y_pred):.4f}")

### 결론

`GridSearchCV`는 다양한 하이퍼파라미터 조합을 체계적으로 탐색하여 모델의 최적 성능을 찾아내는 강력한 도구입니다. 이를 통해 모델의 일반화 성능을 향상시키고, 과대적합 또는 과소적합 문제를 완화할 수 있습니다.

**참고**: `GridSearchCV`는 모든 조합을 탐색하므로 계산 비용이 많이 들 수 있습니다. 더 효율적인 하이퍼파라미터 튜닝 방법으로는 베이지안 최적화(Bayesian Optimization) 기반의 `Optuna`와 같은 라이브러리가 있습니다.